In [3]:
%pip install altair

   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   --------------------------------------- 731.2/731.2 kB 31.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# 1. Import Libraries and Load Data
import pandas as pd
import altair as alt

# Load the Bigfoot data
url = "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv"
df = pd.read_csv(url)

# 2. Process Data for Choropleth Map
# Count sightings per state
state_counts = df['state'].value_counts().reset_index()
state_counts.columns = ['state', 'count']

# Load US states map data
states = alt.topo_feature('https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/us-10m.json', 'states')

# 3. Create Choropleth Map
choropleth = alt.Chart(states).mark_geoshape().encode(
    color='count:Q',
    tooltip=['state:N', 'count:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(state_counts, 'id', ['count', 'state'])
).project('albersUsa')

# 4. Process Data for Line Chart
# Extract year from date
df['year'] = pd.to_datetime(df['date']).dt.year

# Count sightings per state/year
yearly_counts = df.groupby(['state', 'year']).size().reset_index(name='count')

# 5. Create Interactive Line Chart
dropdown = alt.binding_select(options=yearly_counts['state'].unique().tolist(), name='Select State:')
selection = alt.selection_point(
    fields=['state'], 
    bind=dropdown, 
    value=[{'state': 'WA'}]  # Initialize with 'WA' as a list of objects
)

line_chart = alt.Chart(yearly_counts).mark_line().encode(
    x='year:O',
    y='count:Q',
    color=alt.value('steelblue')
).add_params(selection).transform_filter(selection)

# 6. Save the Charts (Run This After Defining Them!)
choropleth.save('assets/choropleth.json')
line_chart.save('assets/line_chart.json')

In [11]:
import pandas as pd

# Manual mapping of state names to codes
state_to_code = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR",
    "California": "CA", "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE",
    "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID",
    "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS",
    "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD",
    "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS",
    "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV",
    "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY",
    "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK",
    "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC",
    "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT",
    "Vermont": "VT", "Virginia": "VA", "Washington": "WA", "West Virginia": "WV",
    "Wisconsin": "WI", "Wyoming": "WY"
}

# Load data
url = "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv"
df = pd.read_csv(url)

# Add state_code column using the manual mapping
df["state_code"] = df["state"].map(state_to_code)

# Save processed data
df.to_csv("bfro_reports_processed.csv", index=False)